In [ ]:
# notebook for first canonical problem for multimodal optimization looking for localized extreme
# events with agent-based military simulation
# Experiment 2.2 - Stochastic, jump discontinuity multimodal localized extrema test problem
# Part 1 - benchmark approaches (MCS, QMC, NEI, TEAD, TuRBO)
#
# Author: Alex Braafladt
# Initial creation: 3/10/23
#
# Goal: Benchmark state-of-the-art Bayesian Optimization approaches (and QMC/MCS) on a multimodal
#       optimization test function focused on localized extrema
#
# Notes:
# -Using the BoTorch framework [1] and NEI from [4]
# -TEAD technique from [2]
# -TuRBO technique from [3]
#
# References:
# [1] M. Balandat et al., “BOTORCH: A framework for efficient Monte-Carlo Bayesian optimization,”
#     Adv. Neural Inf. Process. Syst., vol. 2020-Decem, no. MC, 2020.
# [2] S. Mo et al., “A Taylor Expansion-Based Adaptive Design Strategy for Global Surrogate
#     Modeling With Applications in Groundwater Modeling,” Water Resour. Res., vol. 53, no.
#     12, pp. 10802–10823, 2017, doi: 10.1002/2017WR021622.
# [3] D. Eriksson, M. Pearce, J. R. Gardner, R. Turner, and M. Poloczek, “Scalable global
#     optimization via local Bayesian optimization,” Adv. Neural Inf. Process. Syst., vol.
#     32, no. NeurIPS, 2019.
# [4] B. Letham, B. Karrer, G. Ottoni, and E. Bakshy, “Constrained Bayesian optimization with noisy
#     experiments,” Bayesian Anal., vol. 14, no. 2, pp. 495–519, 2019, doi: 10.1214/18-BA1110.

In [ ]:
# imports
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from extremasearch.globalmm.globalsearch import MultimodalExtremaSearch
from botorch import fit_gpytorch_mll
from botorch.models.gp_regression import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
import numpy as np
import networkx as nx
from botorch.models.transforms import Normalize, Standardize

# setup
dtype = torch.double
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import warnings
from botorch.exceptions import BadInitialCandidatesWarning
warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
# setup file i/o
import os as os
import datetime as dt
# get current working directory
wrkdir = os.getcwd()
print('Current working directory: '+wrkdir)
# set up a data save directory for all future runs
newoutputdir = wrkdir+'\output'
if not os.path.exists(newoutputdir):
    os.makedirs(newoutputdir)
# set up a new directory to store files for the current run - updates at each new full run of notebook
curDatetime = dt.datetime.now()
datasavedir = newoutputdir + r'\\' + 'exp2.2_mme_nonbalanced_benchmark' + str(curDatetime.strftime('%Y%m%d%H%M%S'))
if not os.path.exists(datasavedir):
    os.makedirs(datasavedir)
print('Data save directory: '+datasavedir)

In [ ]:
# canonical problem
def mme_noise_jump_1d_mean(x):
    y = torch.zeros(x.shape)
    y += 0.66*(-1.*x + 0.1*torch.sin(30.*x) + torch.exp(-(100.*(x-0.65))**2.)) * (x >= 0.6)
    y += 0.66*(-1.*x + 0.1*torch.sin(30.*x) - 0.25 + torch.exp(-(100.*(x-0.65))**2.)) * (x < 0.6) * (x >= 0.2)
    y += 0.66*(-1.*x + 0.1*torch.sin(30.*x) + 0.2 + torch.exp(-(100.*(x-0.65))**2.)) * (x < 0.2)
    y += 0.7 * ( torch.exp(-(100.*(x-0.35))**2.) )
    y += 0.4 * ( torch.exp(-(100.*(x-0.92))**2.) )
    y += 0.9
    return y


sigma_max = 0.12
sigma_min = 0.02


def mme_noise_jump_1d(x):
    # mean
    y = mme_noise_jump_1d_mean(x)
    # noise
    noise = torch.zeros(x.shape)
    noise += (torch.randn(x.shape) * sigma_max) * (x >= 0.61) * (x < 0.71)
    noise += (torch.randn(x.shape) * 0.5*sigma_max) * (x >= 0.31) * (x < 0.41)
    noise += (torch.randn(x.shape) * 0.8*sigma_max) * (x >= 0.90) * (x < 0.94)
    noise += (torch.randn(x.shape) * (sigma_max - sigma_min) / (1. + torch.exp(-10.*(x-0.2))) )* (x < 0.61)
    noise += (torch.randn(x.shape) * (sigma_max - sigma_min) / (1. + torch.exp(10.*(x-0.75))) ) * (x >= 0.71)
    # combined
    y_out = y + noise
    # any_neg = y_out < 0
    # if any_neg:
    #     y_out = torch.tensor(0.0)
    return y_out


x1 = torch.linspace(0,1.0,200)
y1 = mme_noise_jump_1d(x1)
fig, ax = plt.subplots(1, 1, figsize=(8,6))
sns.lineplot(x=x1, y=y1, ax=ax, color='r')
plt.savefig(datasavedir + '/'+'full_test_problem_one_realization'+'.png')

In [ ]:
# plot noise bounds as 99% confidence interval
# generate realizations of the function
num_reals = 500
num_x = 200
x1 = torch.linspace(0,1.0,num_x)
y_reps = torch.DoubleTensor(num_x, num_reals)
y_deter = mme_noise_jump_1d_mean(x1)
for i in range(num_reals):
    y_reps[:,i] = mme_noise_jump_1d(x1)

fig, ax = plt.subplots(1, 1, figsize=(8,6))
sns.lineplot(x=x1, y=y_deter, ax=ax)
upper_99 = torch.quantile(y_reps, 0.995, dim=1)
lower_01 = torch.quantile(y_reps, 0.005, dim=1)
ax.fill_between(x1, lower_01, upper_99, color='b', alpha=0.3, label=r'99% quantiles')
plt.savefig(datasavedir + '/'+'full_test_problem_bounds'+'.png')

In [ ]:
def outcome_objective(x):
    """wrapper for the outcome objective function"""
    # return mme_torch(X).type_as(X)
    return mme_noise_jump_1d(x).type_as(x)